In [ ]:
#Basic Server

import socketserver
import threading

class TCPHandler(socketserver.BaseRequestHandler):
    def setup(self):  # 연결 설정
        print("Connected")
        
    def handle(self):  # 데이터 수신
        while True:  # 연결을 유지하려면 무한 루프 실행
            try:
                self.data = self.request.recv(1024).strip()
                if not self.data:
                    return
            except:
                self.finish()
                return
            print("{} wrote:".format(self.client_address[0]))
            print(self.data)

            # 수신 메시지 전송
            self.request.sendall(self.data.upper())
    
    def finish(self):  # 연결 해제
        print("Disconnected")
        self.request.close()

if __name__ == "__main__":
    HOST, PORT = "", 2501
    server = socketserver.TCPServer((HOST, PORT), TCPHandler, bind_and_activate=True)

    # 서버를 별도의 스레드에서 실행
    server_thread = threading.Thread(target=server.serve_forever)
    server_thread.daemon = True  # 메인 스레드 종료 시 함께 종료
    server_thread.start()

    print("Server running in thread", server_thread.name)


OSError: [WinError 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다